In [1]:
import pandas as pd
import numpy as np
from scipy import interpolate

In [2]:
# Funções de reclassificação
def reclass_MB(classe):
    mapping = {
        'Lavouras permanentes': '3.2.2. Perennial Crop',
        'Lavouras - permanentes': '3.2.2. Perennial Crop',
        'Lavouras temporárias': '3.2.1. Temporary Crop',
        'Lavouras - temporárias': '3.2.1. Temporary Crop',
        'Lavouras - área para cultivo de flores': '3.2.1. Temporary Crop',
        'Pastagens - naturais': '2.1_2.2',
        'Pastagens naturais': '2.1_2.2',
        'Pastagens - plantadas em boas condições': '3.1. Pasture',
        'Pastagens - pastagens plantadas em más condições': '3.1. Pasture',
        'Pastagens plantadas': '3.1. Pasture',
        'Sistemas agroflorestais - área cultivada com espécies florestais também usada para lavouras e pastoreio por animais': '3.1. Pasture'
    }
    return mapping.get(classe, None)

def reclass_FAO(classe):
    mapping = {
        'Lavouras - permanentes': '6650 - Land under permanent crops',
        'Lavouras permanentes': '6650 - Land under permanent crops',
        'Lavouras - temporárias': '6621 - Arable land',
        'Lavouras temporárias': '6621 - Arable land',
        'Lavouras - área para cultivo de flores': '6621 - Arable land',
        'Pastagens - naturais': '6659 - Perm. meadows & pastures - Nat. Growing',
        'Pastagens naturais': '6659 - Perm. meadows & pastures - Nat. Growing',
        'Pastagens - plantadas em boas condições': '6656 - Perm. meadows & pastures - Cultivated',
        'Pastagens - pastagens plantadas em más condições': '6656 - Perm. meadows & pastures - Cultivated',
        'Pastagens plantadas': '6656 - Perm. meadows & pastures - Cultivated',
        'Sistemas agroflorestais - área cultivada com espécies florestais também usada para lavouras e pastoreio por animais': '6656 - Perm. meadows & pastures - Cultivated'
    }
    return mapping.get(classe, None)

In [3]:
# Anos para extrapolação
seqano = [str(year) for year in range(2017, 2024)]

# Leitura e processamento do censo 2017
cbr = pd.read_csv('/home/eduardo/Documentos/R/Uso da Terra/inputs/tabela6881.csv', header=5, sep=';', nrows=12, usecols=[4, 5])
cbr.columns = ['Utilização das terras', 'Total']

# Verificar diferença entre total e soma das classes
diff = cbr['Total'].iloc[0] - cbr['Total'].iloc[1:].sum()
print(f"Diferença entre total e soma das classes: {diff}")

# Reclassificação MapBiomas e FAO
cbr['cl_mapbiomas'] = cbr['Utilização das terras'].apply(reclass_MB)
cbr = cbr.dropna(subset=['cl_mapbiomas'])
cbr['cl_fao'] = cbr['Utilização das terras'].apply(reclass_FAO)

# Replicar valores para anos de extrapolação
for ano in seqano:
    cbr[ano] = cbr['Total']

Diferença entre total e soma das classes: -200


In [4]:
# Leitura dados MapBiomas
br2 = pd.read_excel('/home/eduardo/Documentos/R/Uso da Terra/outputs/COBERTURA_MAPBIOMAS_COL9_TM.xlsx', sheet_name='BRASIL2')
br2.loc[br2['class_level_3'].isin(['2.1. Wetland', '2.2. Grassland']), 'class_level_3'] = '2.1_2.2'
br2 = br2.groupby('class_level_3').sum()

# Seleção classes do censo
sbr2 = br2[br2.index.isin(cbr['cl_mapbiomas'].unique())]

# Cálculo taxa de variação
txbr2 = pd.DataFrame(index=sbr2.index)
txbr2['class_level_3'] = txbr2.index
for ano in seqano:
    txbr2[ano] = sbr2[ano] / sbr2['2017']

# Aplicando taxa aos dados do censo
cbr2 = cbr.copy()
for ano in seqano:
    cbr2[ano] = cbr2[ano] * txbr2.loc[cbr2['cl_mapbiomas'].values][ano].values

In [7]:
# Consolidação classes FAO
cbr3 = cbr2.groupby('cl_fao')[seqano].sum().reset_index()
cbr3 = cbr3[cbr3['cl_fao'].notna()].iloc[:-1]  # Remove última linha conforme código original

# Leitura dados históricos
cbr_ant = pd.read_excel('/home/eduardo/Documentos/R/Uso da Terra/inputs/consolidado_censos.xlsx', sheet_name='tab_org_censos', skiprows=4)
cbr_ant = cbr_ant.iloc[:, :-1]  # Remove última coluna conforme código original
cbr_ant = cbr_ant.set_index(cbr_ant.columns[0]).T
cbr_ant.index = cbr_ant.index.astype(str)

# Reclassificação FAO para dados históricos
cbr_ant['cl_fao'] = [reclass_FAO(idx) for idx in cbr_ant.index]
cbr_ant = cbr_ant.dropna(subset=['cl_fao'])
cbr_ant = cbr_ant.sort_values('cl_fao')
cbr_ant.index = cbr_ant['cl_fao']

In [8]:
# Interpolação
anos_hist = [1940, 1950, 1960, 1970, 1975, 1980, 1985, 1995, 2006, 2017]
anos_interp = list(range(1940, 2018))
fao_br = pd.DataFrame(index=cbr_ant['cl_fao'].unique())

for classe in fao_br.index:
    valores_hist = cbr_ant.loc[classe, ['1940', '1950', '1960', '1970', '1975', 
                                      '1980', '1985', '1995', '2006']].astype(float).values
    valor_2017 = cbr3.loc[cbr3['cl_fao'] == classe, '2017'].values[0]
    valores_hist = np.append(valores_hist, valor_2017)
    
    f = interpolate.interp1d(anos_hist, valores_hist)
    fao_br.loc[classe, [str(ano) for ano in anos_interp]] = f(anos_interp)

# Adicionar anos de extrapolação
for ano in seqano[1:]:
    fao_br[ano] = cbr3.set_index('cl_fao')[ano]

KeyError: "None of [Index(['1940', '1950', '1960', '1970', '1975', '1980', '1985', '1995', '2006'], dtype='object', name='Ano')] are in the [index]"